# **PART 5:** Cluster annotation & visualization

In [1]:
import os
# Set the working directory to the repository directory
os.chdir("/home/sebastiaan/PhD/Repositories/book_chapter/")

In [2]:
import pandas as pd

from src.tools import merge_results, vj_parsing

In [25]:
# TCRex results
tcrex = pd.read_csv("./results/tcrex/P1_0.tsv", sep = "\t")
# ClusTCR results
clust = pd.read_csv("./results/clustcr/P1_0_clusters.tsv", sep = "\t")
# Original data
rawdata = pd.read_csv("./data/examples/P1_0.tsv", sep = "\t")

merged_0 = merge_results(
    original = rawdata,
    clusters = clust,
    predictions = tcrex
    )

merged_0 = merged_0.dropna(subset = ["cluster"])

keep = ["duplicate_count",
        "frequency",
        "junction_aa", 
        "v_call", 
        "j_call", 
        "cluster",
        "epitope",
        "pathology",
        "score",
        "bpr"]

merged_0 = merged_0[keep]
merged_0.to_csv("./results/P1_0_clusters_tcrex.tsv", sep = "\t", index = False)

In [26]:
merged_0

,duplicate_count,frequency,junction_aa,v_call,j_call,cluster,epitope,pathology,score,bpr
0,4444.0,1.582432e-03,CASSEREANEQFF,TRBV6-4*00,TRBJ2-1*00,795.0,NaN,NaN,NaN,NaN
1,2356.0,8.389310e-04,CASSDRSGGADEQFF,TRBV6-4*00,TRBJ2-1*00,437.0,NaN,NaN,NaN,NaN
2,789.0,2.809493e-04,CASSYGAGANVLTF,TRBV6-5*00,TRBJ2-6*00,1263.0,NaN,NaN,NaN,NaN
3,737.0,2.624330e-04,CASSEDGNTEAFF,TRBV6-4*00,TRBJ1-1*00,803.0,NaN,NaN,NaN,NaN
4,730.0,2.599404e-04,CASSEATGGANVLTF,TRBV6-4*00,TRBJ2-6*00,494.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
84159,1.0,3.560827e-07,CASWRAYNEQFF,TRBV23-1*00,TRBJ2-1*00,17375.0,NaN,NaN,NaN,NaN
84160,1.0,3.560827e-07,CASSRDGETQYF,TRBV23-1*00,TRBJ2-5*00,17375.0,NaN,NaN,NaN,NaN
84161,1.0,3.560827e-07,CASSRGQELYF,TRBV23-1*00,TRBJ2-7*00,17375.0,NaN,NaN,NaN,NaN
84162,1.0,3.560827e-07,CASKWGDTQYF,TRBV23-1*00,TRBJ2-3*00,17375.0,NaN,NaN,NaN,NaN


In [27]:
# TCRex results
tcrex = pd.read_csv("./results/tcrex/P1_15.tsv", sep = "\t")
# ClusTCR results
clust = pd.read_csv("./results/clustcr/P1_15_clusters.tsv", sep = "\t")
# Original data
rawdata = pd.read_csv("./data/examples/P1_15.tsv", sep = "\t")

merged_15 = merge_results(
    original = rawdata,
    clusters = clust,
    predictions = tcrex
    )

merged_15 = merged_15.dropna(subset = ["cluster"])

keep = ["duplicate_count",
        "frequency",
        "junction_aa", 
        "v_call", 
        "j_call", 
        "cluster",
        "epitope",
        "pathology",
        "score",
        "bpr"]

merged_15 = merged_15[keep]
merged_15.to_csv("./results/P1_15_clusters_tcrex.tsv", sep = "\t", index = False)

In [29]:
merged_15

,duplicate_count,frequency,junction_aa,v_call,j_call,cluster,epitope,pathology,score,bpr
0,1480.0,7.687245e-04,CASSDRSGGADEQFF,TRBV6-4*00,TRBJ2-1*00,243.0,NaN,NaN,NaN,NaN
1,857.0,4.451331e-04,CASSEDPRLQRDEQFF,TRBV6-1*00,TRBJ2-1*00,44.0,NaN,NaN,NaN,NaN
2,687.0,3.568336e-04,CASSDSPTGAYGYTF,TRBV6-1*00,TRBJ1-2*00,267.0,NaN,NaN,NaN,NaN
3,558.0,2.898299e-04,CASSERENTEAFF,TRBV6-4*00,TRBJ1-1*00,607.0,NaN,NaN,NaN,NaN
4,3.0,1.558225e-06,CASSERENTEAFF,TRBV6-1*00,TRBJ1-1*00,607.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
65083,1.0,5.194085e-07,CASHTVGSSETQYF,TRBV21-1*00,TRBJ2-5*00,15525.0,NaN,NaN,NaN,NaN
65084,1.0,5.194085e-07,CARSKTLTGETQYF,TRBV21-1*00,TRBJ2-5*00,15527.0,NaN,NaN,NaN,NaN
65085,1.0,5.194085e-07,CASSKRQTYEQYF,TRBV21-1*00,TRBJ2-7*00,15510.0,NaN,NaN,NaN,NaN
65086,1.0,5.194085e-07,CANSPGRADTQYF,TRBV21-1*00,TRBJ2-3*00,15512.0,NaN,NaN,NaN,NaN


In [59]:
clusters_with_hits = set(merged_0.dropna(subset = ["epitope"]).cluster)
subset = merged_0[merged_0.cluster.isin(clusters_with_hits)]


epi_count = subset.groupby("cluster").count().epitope.sort_index()
cluster_size = merged_0.cluster.value_counts()[clusters_with_hits].sort_index()

res = pd.concat([epi_count, cluster_size], axis = 1)

/tmp/ipykernel_278412/2014800533.py:6: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  cluster_size = merged_0.cluster.value_counts()[clusters_with_hits].sort_index()


In [71]:
import matplotlib.pyplot as plt

res.sort_values(by = "epitope", ascending = False).head(10)
# plt.scatter(res.cluster, res.epitope)

,epitope,cluster
4196.0,112,570
2722.0,84,355
12296.0,75,278
5792.0,63,123
4192.0,53,477
4193.0,41,216
4191.0,28,594
8477.0,27,110
9722.0,26,210
3387.0,25,208
